In [2]:
from pathlib import Path
import sys

# In a notebook, get current directory and go up 2 levels
project_root = Path.cwd().parents[2]

# Add to sys.path if not already present
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
from pyspark.sql.functions import col, lit, create_map
from databricks.connect import DatabricksSession
from src.citibike.citibike_utils import get_trip_duration
from src.utils.datetime_utils import timestamp_to_date_col

['c:\\Users\\DELL\\Desktop\\databricks\\dab_project', 'C:\\Users\\DELL\\Desktop\\databricks\\dab_project', 'c:\\Users\\DELL\\Desktop\\databricks\\dab_project\\citibike_etl\\notebooks\\02_silver', 'C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\DELL\\Desktop\\databricks\\databricks', '', 'c:\\Users\\DELL\\Desktop\\databricks\\databricks\\Lib\\site-packages', 'c:\\Users\\DELL\\Desktop\\databricks\\databricks\\Lib\\site-packages\\win32', 'c:\\Users\\DELL\\Desktop\\databricks\\databricks\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\DELL\\Desktop\\databricks\\databricks\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\DELL\\Desktop\\databricks\\databricks\\Lib\\site-packages\\setuptools\\_vendor']


In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [ ]:
spark = DatabricksSession.builder.getOrCreate()

In [ ]:
df = spark.read.table(f"{catalog}.bronze.jc_citibike")

In [ ]:
df = get_trip_duration(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [ ]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
df = df.withColumn("metadata",
                   create_map(
                       lit("pipeline_id"), lit(pipeline_id),
                       lit("run_id"), lit(run_id),
                       lit("task_id"), lit(task_id), 
                       lit("processed_timestamp"), lit(processed_timestamp)
                   ))

In [ ]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
)

In [ ]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalog}.silver.jc_citibike")